<a href="https://colab.research.google.com/github/shantanu2383/FAMA-FRENCH-5-FACTORS-ST-REVERSAL-LT-REVERSAL-MOMENTUM/blob/main/01_20_FAMA_FRENCH_DAILY_ST_REVERSAL_FACTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps
from sqlite3 import connect
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

conn=connect(':memory:')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=9cdc97e5e6958354366ef9e43767d5ea3acd0df5510ac0f9b47d29d5fb362fe5
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


# IMPORT CLEANED PRICE DATA

In [ ]:
#import cleaned price return data

filepath="/content/gdrive/MyDrive/clean/"

file="cleaned_returns.csv"

daily_returns=pd.read_csv(filepath + file)

# CALCULATE PRIOR DAY RETURN

In [ ]:
daily_returns['date']=pd.to_datetime(daily_returns['date'])

Get the daily return for each day as a percentage and add 1

In [ ]:
daily_returns.drop(columns=['daily_return'], inplace=True)

daily_returns['daily_return']=daily_returns.groupby('ticker')['adj_close'].pct_change() +1

shift the daily return by 1-20 

In [ ]:



for i in range(1, 21):
    daily_returns[f'daily_return_shifted_{i}'] = daily_returns.groupby('ticker')['daily_return'].shift(i)

In [ ]:
daily_returns.dropna(inplace=True)
daily_returns

,Unnamed: 0,ticker,date,adj_close,daily_return,daily_return_shifted_1,daily_return_shifted_2,daily_return_shifted_3,daily_return_shifted_4,daily_return_shifted_5,...,daily_return_shifted_11,daily_return_shifted_12,daily_return_shifted_13,daily_return_shifted_14,daily_return_shifted_15,daily_return_shifted_16,daily_return_shifted_17,daily_return_shifted_18,daily_return_shifted_19,daily_return_shifted_20
21,21,A,2018-02-02,68.728871,0.978306,0.991829,1.006028,0.979337,0.996124,1.012998,...,1.001804,1.011652,0.993029,1.013136,1.000141,0.986345,1.024554,1.002146,1.015988,0.992499
22,22,A,2018-02-05,65.806086,0.957474,0.978306,0.991829,1.006028,0.979337,0.996124,...,1.012190,1.001804,1.011652,0.993029,1.013136,1.000141,0.986345,1.024554,1.002146,1.015988
23,23,A,2018-02-06,66.027947,1.003371,0.957474,0.978306,0.991829,1.006028,0.979337,...,1.005611,1.012190,1.001804,1.011652,0.993029,1.013136,1.000141,0.986345,1.024554,1.002146
24,24,A,2018-02-07,65.651747,0.994302,1.003371,0.957474,0.978306,0.991829,1.006028,...,0.999456,1.005611,1.012190,1.001804,1.011652,0.993029,1.013136,1.000141,0.986345,1.024554
25,25,A,2018-02-08,62.748254,0.955774,0.994302,1.003371,0.957474,0.978306,0.991829,...,1.001906,0.999456,1.005611,1.012190,1.001804,1.011652,0.993029,1.013136,1.000141,0.986345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967008,3967008,ZYXI,2022-12-21,13.750000,1.014012,0.998527,1.019520,0.960346,0.971289,1.046154,...,1.003676,0.985507,1.010989,0.994898,1.017050,0.998520,0.998522,0.977601,1.008012,1.013284
3967009,3967009,ZYXI,2022-12-22,13.600000,0.989091,1.014012,0.998527,1.019520,0.960346,0.971289,...,1.022711,1.003676,0.985507,1.010989,0.994898,1.017050,0.998520,0.998522,0.977601,1.008012
3967010,3967010,ZYXI,2022-12-23,13.810000,1.015441,0.989091,1.014012,0.998527,1.019520,0.960346,...,1.013610,1.022711,1.003676,0.985507,1.010989,0.994898,1.017050,0.998520,0.998522,0.977601
3967011,3967011,ZYXI,2022-12-27,13.760000,0.996379,1.015441,0.989091,1.014012,0.998527,1.019520,...,0.942756,1.013610,1.022711,1.003676,0.985507,1.010989,0.994898,1.017050,0.998520,0.998522


calculate the cumulative product of these returns

In [ ]:
daily_returns['prior_month_ret']=daily_returns['daily_return_shifted_1'] * daily_returns['daily_return_shifted_2'] * daily_returns['daily_return_shifted_3'] * daily_returns['daily_return_shifted_4'] * daily_returns['daily_return_shifted_5'] * daily_returns['daily_return_shifted_6'] * daily_returns['daily_return_shifted_7'] * daily_returns['daily_return_shifted_8'] * daily_returns['daily_return_shifted_9'] * daily_returns['daily_return_shifted_10'] * daily_returns['daily_return_shifted_11'] * daily_returns['daily_return_shifted_12'] * daily_returns['daily_return_shifted_13'] * daily_returns['daily_return_shifted_14'] * daily_returns['daily_return_shifted_15'] * daily_returns['daily_return_shifted_16'] * daily_returns['daily_return_shifted_17'] * daily_returns['daily_return_shifted_18'] * daily_returns['daily_return_shifted_19'] * daily_returns['daily_return_shifted_20'] - 1

In [ ]:
'''
#daily_returns = daily_returns.set_index('date')
daily_returns=daily_returns[daily_returns['daily_return_shifted'].notna()]
daily_returns['daily_return_shifted']=pd.to_numeric(daily_returns['daily_return_shifted'])

daily_returns['prior_month_ret']=daily_returns.groupby('ticker')['daily_return_shifted'].rolling(window=20, min_periods=1).apply(lambda x: np.prod(x)-1)
'''

"\n#daily_returns = daily_returns.set_index('date')\ndaily_returns=daily_returns[daily_returns['daily_return_shifted'].notna()]\ndaily_returns['daily_return_shifted']=pd.to_numeric(daily_returns['daily_return_shifted'])\n\ndaily_returns['prior_month_ret']=daily_returns.groupby('ticker')['daily_return_shifted'].rolling(window=20, min_periods=1).apply(lambda x: np.prod(x)-1)\n"

In [ ]:
for i in range(1, 21):
  daily_returns=daily_returns.drop(columns=[f'daily_return_shifted_{i}'])

In [ ]:
daily_returns.to_csv(filepath +"daily_returns with st returns.csv")

# IMPORT FUNDAMENTALS DATA

Import the fundamentals data so we can match relevant variables to daily returns dataframe

In [ ]:
#import clean fundamentals data
file="cleaned_fundamentals_updated.csv"
fundamentals=pd.read_csv(filepath + file)

In [ ]:
fundamentals.drop(columns=["Unnamed: 0", "index"], inplace=True)

In [ ]:
fundamentals

,ticker,exchange,tot_asset,net_income_loss_share_holder,comm_shares_out,per_end_date,per_type,tot_liab,book_val_per_share,mkt_val,operating_profitability,book_value
0,AAC,NYSE,1002.156,20.3805,31.9080,2021-03-31 00:00:00,Q,997.1564,0.1567,315.89,4.076426,4.9996
1,A,NYSE,7872.000,168.0000,322.0000,2017-01-31 00:00:00,Q,3572.0000,13.3540,NaN,0.039070,4300.0000
2,A,NYSE,8016.000,164.0000,321.0000,2017-04-30 00:00:00,Q,3641.0000,13.6293,NaN,0.037486,4375.0000
3,A,NYSE,8261.000,175.0000,322.0000,2017-07-31 00:00:00,Q,3650.0000,14.3199,NaN,0.037953,4611.0000
4,A,NYSE,8426.000,177.0000,322.0000,2017-10-31 00:00:00,Q,3591.0000,15.0155,NaN,0.036608,4835.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
107192,ZYXI,NASDAQ,90.723,6.1070,38.2514,2021-09-30 00:00:00,Q,27.3400,1.6570,NaN,0.096351,63.3830
107193,ZYXI,NASDAQ,133.274,8.8940,39.7380,2021-12-31 00:00:00,Q,59.3490,1.8603,NaN,0.120311,73.9250
107194,ZYXI,NASDAQ,132.041,1.3770,39.7770,2022-03-31 00:00:00,Q,56.2230,1.9061,NaN,0.018162,75.8180
107195,ZYXI,NASDAQ,119.575,3.3460,38.4040,2022-06-30 00:00:00,Q,50.5660,1.7969,NaN,0.048486,69.0090


Our aim here is to match the market cap of the last quarter for each ticker to each daily date for each ticker (to avoid look ahead bias). This is method 1 of calculating daily market cap. We will also try a different method where we multiply the closing price by shares outstanding at the end of the quarter.

In [ ]:
fundamentals['per_end_date']=pd.to_datetime(fundamentals['per_end_date'], format="%Y-%m-%d")
daily_returns['date']=pd.to_datetime(daily_returns['date'], format="%Y-%m-%d")
#set both dates to relevant datetime format

#get the specific quarter from daily returns data--> We will subtract one quarter
#from the date, then extract the quarter

one_quarter = pd.tseries.offsets.DateOffset(months=3)
daily_returns['last_quarter']=daily_returns['date']-one_quarter
daily_returns['last_quarter']=pd.PeriodIndex(daily_returns.last_quarter, freq='Q')

#extract quarter from daily returns dataframe
fundamentals['quarter']=pd.PeriodIndex(fundamentals.per_end_date, freq='Q')


In [ ]:
daily_returns.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
#we now merge the dataframe together using SQL

#apply strings to quarters for SQL
fundamentals['quarter']=fundamentals['quarter'].apply(str)
daily_returns['last_quarter']=daily_returns['last_quarter'].apply(str)

#send to sql
daily_returns.to_sql('dr', conn, if_exists='replace')
fundamentals.to_sql('f', conn, if_exists='replace')

#query


query='''SELECT DISTINCT dr.*, f.mkt_val, f.exchange
          FROM dr
          LEFT JOIN f ON dr.last_quarter=f.quarter AND dr.ticker=f.ticker
          '''

daily_returns_size=pd.read_sql(query, conn)

In [ ]:
daily_returns_size.drop(columns=["index"], inplace=True)

In [ ]:
daily_returns=daily_returns_size

In [ ]:
daily_returns

,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange
0,A,2018-02-02 00:00:00,68.728871,0.978306,0.050635,2017Q4,NaN,NYSE
1,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE
2,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE
3,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE
4,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE
...,...,...,...,...,...,...,...,...
3880602,ZYXI,2022-12-21 00:00:00,13.750000,1.014012,0.000738,2022Q3,NaN,NASDAQ
3880603,ZYXI,2022-12-22 00:00:00,13.600000,0.989091,0.001457,2022Q3,NaN,NASDAQ
3880604,ZYXI,2022-12-23 00:00:00,13.810000,1.015441,-0.017341,2022Q3,NaN,NASDAQ
3880605,ZYXI,2022-12-27 00:00:00,13.760000,0.996379,0.020695,2022Q3,NaN,NASDAQ


#METHOD 2 OF CALCULATING DAILY MARKET CAP

In this section we extract the common shares outstanding at the end of each quarter from Fundamental data. We then merge these into our daily returns dataframe, so we will have the shares oustanding for the previous quarter for each date. This allows us to calculate an alternate measure of market cap as (close_price * (shares outstanding at last quarter). By doing this, this measure of market cap will be able to vary by day, which makes them more relevant for calculating daily size portfolios. THIS IS A PROXY FOR DAILY MARKET CAPITALISATION

In [ ]:

filepath="/content/gdrive/MyDrive/ZACKS_SHRS_2_8f681e89a68dd3916ae2d06a8af3584f.csv"


f_shares=pd.read_csv(filepath)

In [ ]:
f_shares

,ticker,m_ticker,comp_name,fye,per_type,per_end_date,active_ticker_flag,shares_out,avg_d_shares
0,A,A2,AGILENT TECH,10,Q,2022-07-31,Y,296.04,299.0
1,A,A2,AGILENT TECH,10,Q,2022-04-30,Y,298.71,301.0
2,A,A2,AGILENT TECH,10,Q,2022-01-31,Y,300.37,303.0
3,A,A2,AGILENT TECH,10,Q,2021-10-31,Y,302.00,305.0
4,A,A2,AGILENT TECH,10,Q,2021-07-31,Y,302.72,306.0
...,...,...,...,...,...,...,...,...,...
343329,ZZ,ZZ,SEALY CORP,11,Q,2018-02-28,N,NaN,NaN
343330,ZZ,ZZ,SEALY CORP,11,Q,2017-11-30,N,NaN,NaN
343331,ZZ,ZZ,SEALY CORP,11,Q,2017-08-31,N,NaN,NaN
343332,ZZ,ZZ,SEALY CORP,11,Q,2017-05-31,N,NaN,NaN


In [ ]:
f_shares['quarter']=pd.PeriodIndex(f_shares.per_end_date, freq='Q')

In [ ]:
f_shares['quarter']=f_shares['quarter'].apply(str)

f_shares.to_sql("f_shares", conn, if_exists="replace")
daily_returns.to_sql("dr", conn, if_exists="replace")

query='''
        SELECT DISTINCT dr.*, f_shares.shares_out
          FROM dr
          LEFT JOIN f_shares ON dr.last_quarter=f_shares.quarter AND dr.ticker=f_shares.ticker '''

daily_returns_shares=pd.read_sql(query, conn)

In [ ]:
daily_returns_shares

,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out
0,0,A,2018-02-02 00:00:00,68.728871,0.978306,0.050635,2017Q4,NaN,NYSE,321.83
1,1,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83
2,2,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83
3,3,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83
4,4,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83
...,...,...,...,...,...,...,...,...,...,...
3880602,3880602,ZYXI,2022-12-21 00:00:00,13.750000,1.014012,0.000738,2022Q3,NaN,NASDAQ,NaN
3880603,3880603,ZYXI,2022-12-22 00:00:00,13.600000,0.989091,0.001457,2022Q3,NaN,NASDAQ,NaN
3880604,3880604,ZYXI,2022-12-23 00:00:00,13.810000,1.015441,-0.017341,2022Q3,NaN,NASDAQ,NaN
3880605,3880605,ZYXI,2022-12-27 00:00:00,13.760000,0.996379,0.020695,2022Q3,NaN,NASDAQ,NaN


In [ ]:
daily_returns=daily_returns_shares

#calculate market cap by multiplying closing price by shares outstanding

daily_returns['mkt_cap_2']=daily_returns['adj_close'].mul(daily_returns['shares_out'])
#set market cap to previous day market cap

daily_returns['mkt_cap_2']=daily_returns['mkt_cap_2'].shift(1)

In [ ]:
daily_returns

,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2
0,0,A,2018-02-02 00:00:00,68.728871,0.978306,0.050635,2017Q4,NaN,NYSE,321.83,NaN
1,1,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687
2,2,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568
3,3,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294
4,4,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803
...,...,...,...,...,...,...,...,...,...,...,...
3880602,3880602,ZYXI,2022-12-21 00:00:00,13.750000,1.014012,0.000738,2022Q3,NaN,NASDAQ,NaN,NaN
3880603,3880603,ZYXI,2022-12-22 00:00:00,13.600000,0.989091,0.001457,2022Q3,NaN,NASDAQ,NaN,NaN
3880604,3880604,ZYXI,2022-12-23 00:00:00,13.810000,1.015441,-0.017341,2022Q3,NaN,NASDAQ,NaN,NaN
3880605,3880605,ZYXI,2022-12-27 00:00:00,13.760000,0.996379,0.020695,2022Q3,NaN,NASDAQ,NaN,NaN


# DAILY RETURN SORT

In this section we sort on daily prior returns:
The methodology is as follows:



1.   Get subset of NYSE stocks only
2.   Calculate breakpoint of NYSE stocks for each date
3.   Sort into prior return portfolios based on prior daily return cf breakpoints



In [ ]:
#filter for relevant exchanges
daily_returns=daily_returns[(daily_returns['exchange']=="NYSE") | (daily_returns['exchange']=="AMEX") | (daily_returns['exchange']=="NASDAQ")]
daily_returns['date']=pd.to_datetime(daily_returns['date'])



<ipython-input-39-e8f48d7a8fce>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_returns['date']=pd.to_datetime(daily_returns['date'])


In [ ]:
'''FROM FAMA FRENCH

The six portfolios used to construct Mom each day include NYSE, AMEX, and NASDAQ stocks with prior return data. 
To be included in a portfolio for day t (formed at the end of day t-1), a stock must have a price for the end of day t-251 and a good return for t-21. 
In addition, any missing returns from t-250 to t-22 must be -99.0, CRSP's code for a missing price. 
Each included stock also must have ME for the end of day t-1.
'''

"FROM FAMA FRENCH\n\nThe six portfolios used to construct Mom each day include NYSE, AMEX, and NASDAQ stocks with prior return data. \nTo be included in a portfolio for day t (formed at the end of day t-1), a stock must have a price for the end of day t-251 and a good return for t-21. \nIn addition, any missing returns from t-250 to t-22 must be -99.0, CRSP's code for a missing price. \nEach included stock also must have ME for the end of day t-1.\n"

In [ ]:
daily_returns

,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2
0,0,A,2018-02-02,68.728871,0.978306,0.050635,2017Q4,NaN,NYSE,321.83,NaN
1,1,A,2018-02-05,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687
2,2,A,2018-02-06,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568
3,3,A,2018-02-07,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294
4,4,A,2018-02-08,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803
...,...,...,...,...,...,...,...,...,...,...,...
3880602,3880602,ZYXI,2022-12-21,13.750000,1.014012,0.000738,2022Q3,NaN,NASDAQ,NaN,NaN
3880603,3880603,ZYXI,2022-12-22,13.600000,0.989091,0.001457,2022Q3,NaN,NASDAQ,NaN,NaN
3880604,3880604,ZYXI,2022-12-23,13.810000,1.015441,-0.017341,2022Q3,NaN,NASDAQ,NaN,NaN
3880605,3880605,ZYXI,2022-12-27,13.760000,0.996379,0.020695,2022Q3,NaN,NASDAQ,NaN,NaN


In [ ]:
daily_returns_sorts= daily_returns.dropna(subset=['prior_month_ret', 'mkt_cap_2'])

In [ ]:
daily_returns_sorts

,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2
1,1,A,2018-02-05,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687
2,2,A,2018-02-06,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568
3,3,A,2018-02-07,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294
4,4,A,2018-02-08,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803
5,5,A,2018-02-09,64.609962,1.029669,-0.081085,2017Q4,NaN,NYSE,321.83,20194.270530
...,...,...,...,...,...,...,...,...,...,...,...
3879648,3879648,ZYNE,2022-12-22,0.540000,0.929432,0.096226,2022Q3,33.49,NASDAQ,45.75,26.580750
3879649,3879649,ZYNE,2022-12-23,0.539900,0.999815,0.009912,2022Q3,33.49,NASDAQ,45.75,24.705000
3879650,3879650,ZYNE,2022-12-27,0.500000,0.926097,-0.036237,2022Q3,33.49,NASDAQ,45.75,24.700425
3879651,3879651,ZYNE,2022-12-28,0.515700,1.031400,-0.161496,2022Q3,33.49,NASDAQ,45.75,22.875000


In [ ]:
#filter for NYSE stocks only
return_breakpoints=daily_returns_sorts[daily_returns['exchange']=="NYSE"]


<ipython-input-44-18c4d46165cd>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return_breakpoints=daily_returns_sorts[daily_returns['exchange']=="NYSE"]


In [ ]:

q30_ret=return_breakpoints.groupby('date').apply(lambda x: np.quantile(x.prior_month_ret, 0.3))
q70_ret=return_breakpoints.groupby('date').apply(lambda x: np.quantile(x.prior_month_ret, 0.7))



In [ ]:
#format these values as dataframes to match together

q30_ret=pd.DataFrame(q30_ret)
q30_ret=q30_ret.rename(columns={0: 'q30'})

q70_ret=pd.DataFrame(q70_ret)
q70_ret=q70_ret.rename(columns={0: 'q70'})

In [ ]:
q30_ret

,q30
date,
2018-02-02,-0.014761
2018-02-05,-0.045778
2018-02-06,-0.082924
2018-02-07,-0.072480
2018-02-08,-0.069319
...,...
2022-12-21,-0.073864
2022-12-22,-0.073404
2022-12-23,-0.091675


In [ ]:
return_quantiles=pd.merge(q30_ret, q70_ret, on='date')

In [ ]:
return_quantiles

,q30,q70
date,,
2018-02-02,-0.014761,0.057277
2018-02-05,-0.045778,0.027244
2018-02-06,-0.082924,-0.014383
2018-02-07,-0.072480,-0.005190
2018-02-08,-0.069319,-0.001807
...,...,...
2022-12-21,-0.073864,0.004384
2022-12-22,-0.073404,0.005027
2022-12-23,-0.091675,-0.000498


We now merge these NYSE return_quantiles back into our original dataframe  

In [ ]:
daily_returns

daily_returns_sorts.to_sql("dr", conn, if_exists='replace')
return_quantiles.to_sql("q", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT dr.*, q.q30 as return_q30, q.q70 as return_q70
        FROM dr
        LEFT JOIN q
        ON dr.date=q.date'''

In [ ]:
daily_returns_sorts_quantiles=pd.read_sql(query, conn)

In [ ]:
daily_returns_sorts=daily_returns_sorts_quantiles

In [ ]:
import numpy as np

'''
daily_returns_sorts['return_portfolio'] = np.where(daily_returns_sorts['prior_month_ret'] > daily_returns_sorts['return_q70'], 'H',
                                           np.where(daily_returns_sorts['prior_month_ret'] <= daily_returns_sorts['return_q70'] , 'M',
                                                    np.where(daily_returns_sorts['return_q30'] > daily_returns_sorts['prior_month_ret'],'L', np.nan)))
'''

daily_returns_sorts['return_portfolio']=np.where(daily_returns_sorts['prior_month_ret'] > daily_returns_sorts['return_q70'], 'H', np.where(daily_returns_sorts['return_q30']> daily_returns_sorts['prior_month_ret'], 'L', 'M'))


In [ ]:
daily_returns_sorts

,level_0,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2,return_q30,return_q70,return_portfolio
0,1,1,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687,-0.045778,0.027244,H
1,2,2,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568,-0.082924,-0.014383,M
2,3,3,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294,-0.072480,-0.005190,M
3,4,4,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803,-0.069319,-0.001807,M
4,5,5,A,2018-02-09 00:00:00,64.609962,1.029669,-0.081085,2017Q4,NaN,NYSE,321.83,20194.270530,-0.092648,-0.027115,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701852,3879648,3879648,ZYNE,2022-12-22 00:00:00,0.540000,0.929432,0.096226,2022Q3,33.49,NASDAQ,45.75,26.580750,-0.073404,0.005027,H
3701853,3879649,3879649,ZYNE,2022-12-23 00:00:00,0.539900,0.999815,0.009912,2022Q3,33.49,NASDAQ,45.75,24.705000,-0.091675,-0.000498,H
3701854,3879650,3879650,ZYNE,2022-12-27 00:00:00,0.500000,0.926097,-0.036237,2022Q3,33.49,NASDAQ,45.75,24.700425,-0.085593,0.000889,M
3701855,3879651,3879651,ZYNE,2022-12-28 00:00:00,0.515700,1.031400,-0.161496,2022Q3,33.49,NASDAQ,45.75,22.875000,-0.070436,0.007913,L


In [ ]:
daily_returns_sorts

,level_0,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2,return_q30,return_q70,return_portfolio
0,1,1,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687,-0.045778,0.027244,H
1,2,2,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568,-0.082924,-0.014383,M
2,3,3,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294,-0.072480,-0.005190,M
3,4,4,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803,-0.069319,-0.001807,M
4,5,5,A,2018-02-09 00:00:00,64.609962,1.029669,-0.081085,2017Q4,NaN,NYSE,321.83,20194.270530,-0.092648,-0.027115,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701852,3879648,3879648,ZYNE,2022-12-22 00:00:00,0.540000,0.929432,0.096226,2022Q3,33.49,NASDAQ,45.75,26.580750,-0.073404,0.005027,H
3701853,3879649,3879649,ZYNE,2022-12-23 00:00:00,0.539900,0.999815,0.009912,2022Q3,33.49,NASDAQ,45.75,24.705000,-0.091675,-0.000498,H
3701854,3879650,3879650,ZYNE,2022-12-27 00:00:00,0.500000,0.926097,-0.036237,2022Q3,33.49,NASDAQ,45.75,24.700425,-0.085593,0.000889,M
3701855,3879651,3879651,ZYNE,2022-12-28 00:00:00,0.515700,1.031400,-0.161496,2022Q3,33.49,NASDAQ,45.75,22.875000,-0.070436,0.007913,L


# Daily Market Cap Sorts

In this section we calculate the daily market cap portfolios. The methodology is as follows:



1.   Assign 'mkt_val" to empty 'mkt_val_2' observations
2.   Get subset of NYSE stocks only
3.   Calculate breakpoint of NYSE stocks for each date
4.   Sort into prior return portfolios based on prior daily return cf breakpoints



In [ ]:
daily_returns_sorts

,level_0,index,ticker,date,adj_close,daily_return,prior_month_ret,last_quarter,mkt_val,exchange,shares_out,mkt_cap_2,return_q30,return_q70,return_portfolio
0,1,1,A,2018-02-05 00:00:00,65.806086,0.957474,0.035610,2017Q4,NaN,NYSE,321.83,22119.012687,-0.045778,0.027244,H
1,2,2,A,2018-02-06 00:00:00,66.027947,1.003371,-0.024034,2017Q4,NaN,NYSE,321.83,21178.372568,-0.082924,-0.014383,M
2,3,3,A,2018-02-07 00:00:00,65.651747,0.994302,-0.022841,2017Q4,NaN,NYSE,321.83,21249.774294,-0.072480,-0.005190,M
3,4,4,A,2018-02-08 00:00:00,62.748254,0.955774,-0.051693,2017Q4,NaN,NYSE,321.83,21128.701803,-0.069319,-0.001807,M
4,5,5,A,2018-02-09 00:00:00,64.609962,1.029669,-0.081085,2017Q4,NaN,NYSE,321.83,20194.270530,-0.092648,-0.027115,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701852,3879648,3879648,ZYNE,2022-12-22 00:00:00,0.540000,0.929432,0.096226,2022Q3,33.49,NASDAQ,45.75,26.580750,-0.073404,0.005027,H
3701853,3879649,3879649,ZYNE,2022-12-23 00:00:00,0.539900,0.999815,0.009912,2022Q3,33.49,NASDAQ,45.75,24.705000,-0.091675,-0.000498,H
3701854,3879650,3879650,ZYNE,2022-12-27 00:00:00,0.500000,0.926097,-0.036237,2022Q3,33.49,NASDAQ,45.75,24.700425,-0.085593,0.000889,M
3701855,3879651,3879651,ZYNE,2022-12-28 00:00:00,0.515700,1.031400,-0.161496,2022Q3,33.49,NASDAQ,45.75,22.875000,-0.070436,0.007913,L


In [ ]:
missing_values = daily_returns_sorts['mkt_cap_2'].isna().sum()
print("Number of missing values in mkt_cap_2 column:", missing_values)

Number of missing values in mkt_cap_2 column: 0


Get subset of NYSE stocks

In [ ]:
size_breakpoints=daily_returns_sorts[daily_returns_sorts['exchange']=="NYSE"]

In [ ]:
median_size=size_breakpoints.groupby('date').apply(lambda x: np.quantile(x.mkt_cap_2, 0.5))


In [ ]:
median_size=pd.DataFrame(median_size)


median_size=median_size.rename(columns={0: 'median_size'})

In [ ]:
daily_returns_sorts.drop(columns=["level_0", "index"], inplace=True)

In [ ]:
daily_returns_sorts.to_sql('dr', conn, if_exists='replace')
median_size.to_sql('m', conn, if_exists='replace')

query=''' 
        SELECT DISTINCT dr.*, m.median_size
          FROM dr
          LEFT JOIN m ON dr.date=m.date'''

daily_returns_sorts_size=pd.read_sql(query, conn)

In [ ]:
daily_returns_sorts_size

In [ ]:
daily_returns_sorts_size['size_portfolio']=np.where(daily_returns_sorts_size['mkt_cap_2']>daily_returns_sorts_size['median_size'], 'B', 'S')

In [ ]:
daily_returns_sorts_size

In [ ]:
daily_returns_portfolios=daily_returns_sorts_size
daily_returns_portfolios.to_csv(filepath + "0120-daily_returns_portfolios.csv")

# Portfolio Construction

In [ ]:
daily_returns_portfolios

In [ ]:
#parse relevant variables

daily_returns_portfolios.to_sql('p', conn, if_exists='replace')

query=''' 
        SELECT DISTINCT p.ticker, p.date, p.adj_close, p.daily_return, p.prior_month_ret as prior_daily_ret, p.mkt_cap_2 as mkt_val, p.return_portfolio, p.size_portfolio
          FROM p
        '''

portfolio_construction=pd.read_sql(query, conn)

In [ ]:
main=portfolio_construction
main

Here we group the main dataframe by date, size portfolio and return portfolio. We apply a lambda function to calculate the average weighted return on that day for each of our portfolios.

In [ ]:
valueWeightRet=main.groupby(['date', 'size_portfolio', 'return_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['daily_return']), weights=pd.to_numeric(x['mkt_val'])))


convert this into a dataframe and reset the index so we get the portfolios as columns

In [ ]:
valueWeightRet_df=pd.DataFrame(valueWeightRet)
valueWeightRet_df=valueWeightRet_df.reset_index(level=['size_portfolio', 'return_portfolio'])

In [ ]:
portfolios=valueWeightRet_df

In [ ]:
portfolios[0]=portfolios[0]-1
portfolios[0]*=100

In [ ]:
portfolios

,size_portfolio,return_portfolio,0
date,,,
2018-02-02 00:00:00,B,H,-2.080675
2018-02-02 00:00:00,B,L,-2.776649
2018-02-02 00:00:00,B,M,-2.183604
2018-02-02 00:00:00,S,H,-2.556602
2018-02-02 00:00:00,S,L,-2.459696
...,...,...,...
2022-12-28 00:00:00,B,L,-1.551114
2022-12-28 00:00:00,B,M,-1.113969
2022-12-28 00:00:00,S,H,-1.175055


In [ ]:
x=['size_portfolio', 'return_portfolio']

for x in x:
  portfolios[x]=portfolios[x].apply(str)

portfolios['size_return_portfolio']=portfolios['size_portfolio'] + "/" + portfolios['return_portfolio'] 

portfolios=portfolios.drop(['size_portfolio', 'return_portfolio'], axis=1)

portfolios=pd.pivot(portfolios, columns='size_return_portfolio', values=0)

In [ ]:
portfolios

size_return_portfolio,B/H,B/L,B/M,S/H,S/L,S/M
date,,,,,,
2018-02-02 00:00:00,-2.080675,-2.776649,-2.183604,-2.556602,-2.459696,-2.249046
2018-02-05 00:00:00,-4.023395,-3.347674,-4.234039,-3.622975,-3.421723,-3.461550
2018-02-06 00:00:00,1.744482,3.151920,1.620917,1.263050,2.188935,0.781918
2018-02-07 00:00:00,-0.130018,-0.914540,-0.890780,0.197880,-0.119044,0.211138
2018-02-08 00:00:00,-4.174559,-3.023809,-3.677326,-3.223875,-2.914498,-2.599270
...,...,...,...,...,...,...
2022-12-21 00:00:00,1.537599,1.591356,1.345724,1.369655,1.265852,1.460155
2022-12-22 00:00:00,-1.017333,-2.369773,-1.538792,-0.911411,-1.473325,-1.190592
2022-12-23 00:00:00,0.149774,0.646185,0.522852,-0.189039,0.035728,0.195195


In [ ]:
portfolios
x=['B/H', 'B/L', 'B/M', 'S/H', 'S/L', 'S/M']
for x in x:
  portfolios[x]=pd.to_numeric(portfolios[x], errors='coerce')


In [ ]:
portfolios['ST_rev']=0.5 * (portfolios['S/L'] + portfolios['B/L']) - 0.5 * (portfolios['S/H']+portfolios['B/H'])

In [ ]:
portfolios

size_return_portfolio,B/H,B/L,B/M,S/H,S/L,S/M,ST_rev
date,,,,,,,
2018-02-02 00:00:00,-2.080675,-2.776649,-2.183604,-2.556602,-2.459696,-2.249046,-0.299534
2018-02-05 00:00:00,-4.023395,-3.347674,-4.234039,-3.622975,-3.421723,-3.461550,0.438486
2018-02-06 00:00:00,1.744482,3.151920,1.620917,1.263050,2.188935,0.781918,1.166662
2018-02-07 00:00:00,-0.130018,-0.914540,-0.890780,0.197880,-0.119044,0.211138,-0.550723
2018-02-08 00:00:00,-4.174559,-3.023809,-3.677326,-3.223875,-2.914498,-2.599270,0.730064
...,...,...,...,...,...,...,...
2022-12-21 00:00:00,1.537599,1.591356,1.345724,1.369655,1.265852,1.460155,-0.025023
2022-12-22 00:00:00,-1.017333,-2.369773,-1.538792,-0.911411,-1.473325,-1.190592,-0.957177
2022-12-23 00:00:00,0.149774,0.646185,0.522852,-0.189039,0.035728,0.195195,0.360589


# Import Fama French Data for comparison



In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
datasets = get_available_datasets()
datasets

In [ ]:

FF_ST_Factor=[dataset for dataset in datasets if 'F-F_ST_Reversal_Factor_daily' in dataset ]

In [ ]:
FF_ST_Factor=web.DataReader(FF_ST_Factor[0],'famafrench',start='2017-11-01',end='2023-01-01')[0]

In [ ]:
FF_ST_Factor

,ST_Rev
Date,
2017-11-01,-0.08
2017-11-02,-0.52
2017-11-03,-0.33
2017-11-06,-0.21
2017-11-07,0.08
...,...
2022-11-23,0.85
2022-11-25,0.03
2022-11-28,0.38


In [ ]:
portfolios

In [ ]:
FF_ST_Factor['Date']=FF_ST_Factor.index
FF_ST_Factor['Date']=FF_ST_Factor['Date'].apply(str)
FF_ST_Factor['Date']=pd.to_datetime(FF_ST_Factor['Date'], format="%Y-%m")

In [ ]:
FF_ST_Factorr=FF_ST_Factor.set_index('Date', drop=True)
FF_ST_Factor.drop(columns=["Date"], axis=1, inplace=True)
portfolios.to_sql('p', conn, if_exists='replace')
FF_ST_Factor.to_sql('c', conn, if_exists='replace')

In [ ]:
query=''' 
        SELECT DISTINCT p.date, p.ST_rev, c.ST_Rev as ST_rev_FF
        FROM p
        LEFT JOIN c ON p.date=c.Date
         '''

checker=pd.read_sql(query, conn)

In [ ]:
checker


,date,ST_rev,ST_rev_FF
0,2018-02-02 00:00:00,-0.299534,-0.43
1,2018-02-05 00:00:00,0.438486,0.52
2,2018-02-06 00:00:00,1.166662,0.92
3,2018-02-07 00:00:00,-0.550723,-0.88
4,2018-02-08 00:00:00,0.730064,0.56
...,...,...,...
1230,2022-12-21 00:00:00,-0.025023,NaN
1231,2022-12-22 00:00:00,-0.957177,NaN
1232,2022-12-23 00:00:00,0.360589,NaN
1233,2022-12-27 00:00:00,-1.674791,NaN


In [ ]:
checker.dropna(inplace=True)

In [ ]:
checker.corr(method='pearson')

,ST_rev,ST_rev_FF
ST_rev,1.000000,0.977328
ST_rev_FF,0.977328,1.000000


In [ ]:
checker.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

,ST_rev,ST_rev_FF
count,1216.000000,1216.000000
mean,0.037870,0.038051
std,1.265928,1.268097
min,-10.073775,-9.140000
10%,-0.982327,-1.000000
25%,-0.463217,-0.470000
50%,-0.054487,-0.050000
75%,0.437633,0.450000
90%,1.127476,1.170000
max,12.460334,12.290000


In [ ]:
FF_ST_Factor.drop(columns=["Date"], axis=1, inplace=True)

In [ ]:
clean_ff_daily_st_reversal=checker

In [ ]:
filepath="/content/gdrive/MyDrive/"
file="clean_ff_daily_st_reversal"

In [ ]:
clean_ff_daily_st_reversal.to_csv(filepath + file)